In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from mixer import libbgmg

ld=np.loadtxt('mixer-test/tiny/chr21qc.ld.gz'); r2=np.multiply(ld, ld); r4=np.multiply(r2, r2)
frq=pd.read_csv('mixer-test/tiny/chr21qc.frq', delim_whitespace=True)  # nsubj=100, nchr=200 => 3 digits precision is accurate
bim=pd.read_csv('mixer-test/tiny/chr21qc.bim', delim_whitespace=True, header=None, names='CHR SNP GP BP A1 A2'.split())

lib=libbgmg.LibBgmg('../src/build/lib/libbgmg.so', init_log='mixer-test/test.log', context_id=0, dispose=True)
lib.defvec = bim['CHR'].notnull()
lib.chrnumvec = bim['CHR'].astype(int).values

r2min = 0.0499 # 0.05
ldscore_r2min = 0.001 # 0.001

lib.calc_ld_matrix(bfile='mixer-test/tiny/chr21qc', outfile='mixer-test/tiny/chr21qc.mixer.ld', r2min=r2min, ldscore_r2min=ldscore_r2min, ld_window=0, ld_window_kb=0)
lib.set_ld_r2_coo_from_file(21, 'mixer-test/tiny/chr21qc.mixer.ld')
lib.set_ld_r2_csr()
lib_hetvec = 2*np.multiply(lib.mafvec, 1-lib.mafvec)
hetvec = 2*np.multiply(frq.MAF, 1-frq.MAF)
r2_het = np.multiply(r2, np.tile(hetvec, (len(hetvec), 1)))
r4_het = np.multiply(r2_het, r2_het)

# check that MAF is OK, except that libbgmg MAF is opposite (e.g. about A2, not about A1)
print('max MAF difference:', np.max(np.abs(1-lib.mafvec) - frq.MAF))
print('max HET difference:', np.max(np.abs(lib_hetvec - hetvec)))

lib.set_option('retrieve_ld_sum_type', 0); ld_sum_r2_above = lib.ld_sum_r2
lib.set_option('retrieve_ld_sum_type', 1); ld_sum_r2_below = lib.ld_sum_r2
lib.set_option('retrieve_ld_sum_type', 2); ld_sum_r2_adjust_for_hvec_above = lib.ld_sum_r2
lib.set_option('retrieve_ld_sum_type', 3); ld_sum_r2_adjust_for_hvec_below = lib.ld_sum_r2
lib.set_option('retrieve_ld_sum_type', 0); ld_sum_r4_above = lib.ld_sum_r4
lib.set_option('retrieve_ld_sum_type', 2); ld_sum_r4_adjust_for_hvec_above = lib.ld_sum_r4

print('max diff in r2_sum_above: ', np.max(np.abs(np.sum(np.multiply(r2, r2 >= r2min), 1) - ld_sum_r2_above)))
print('max diff in r2_sum_below: ', np.max(np.abs(np.sum(np.multiply(r2, (r2 < r2min) & (r2 >= ldscore_r2min) ), 1) - ld_sum_r2_below)))
print('max diff in r2_sum_total: ', np.max(np.abs(np.sum(np.multiply(r2, (r2 >= ldscore_r2min) ), 1) - (ld_sum_r2_above + ld_sum_r2_below) )))
print('max diff in r4_sum_above: ', np.max(np.abs(np.sum(np.multiply(r4, r2 >= r2min), 1) - ld_sum_r4_above)))

print('max diff in r2_sum_het_above: ', np.max(np.abs(np.sum(np.multiply(r2_het, r2 >= r2min), 1) - ld_sum_r2_adjust_for_hvec_above)))
print('max diff in r2_sum_het_below: ', np.max(np.abs(np.sum(np.multiply(r2_het, (r2 < r2min) & (r2 >= ldscore_r2min) ), 1) - ld_sum_r2_adjust_for_hvec_below)))
print('max diff in r2_sum_het_total: ', np.max(np.abs(np.sum(np.multiply(r2_het, (r2 >= ldscore_r2min) ), 1) - (ld_sum_r2_adjust_for_hvec_above + ld_sum_r2_adjust_for_hvec_below) )))
print('max diff in r4_sum_het_above: ', np.max(np.abs(np.sum(np.multiply(r4_het, r2 >= r2min), 1) - ld_sum_r4_adjust_for_hvec_above)))

[snp, tag, lib_r] = lib.get_ld_r2_chr(21)
lib_r_mat = coo_matrix((lib_r, (snp, tag)), shape=(lib.num_snp, lib.num_tag)).toarray()
lib_r2 = np.multiply(lib_r_mat, lib_r_mat)

r2_above = np.multiply(ld, ld); r2_above = np.multiply(r2_above, r2_above >= r2min);
print('max diff in r2 sparse matrix: ', np.max(np.abs(lib_r2 - r2_above)))


max MAF difference: 2.8610229518832853e-08
max HET difference: 5.4162740702190515e-08
max diff in r2_sum_above:  0.0009558380511123232
max diff in r2_sum_below:  0.0009984926233030933
max diff in r2_sum_total:  0.0012759102995758553
max diff in r4_sum_above:  0.0014510578888824455
max diff in r2_sum_het_above:  0.00021826482092990318
max diff in r2_sum_het_below:  0.0004796392157384588
max diff in r2_sum_het_total:  0.0005579415790659681
max diff in r4_sum_het_above:  0.0001568508609040009
max diff in r2 sparse matrix:  3.027609337069581e-05


In [11]:
lib_r2.shape

(861, 861)

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from mixer import libbgmg

bim=pd.read_csv('/home/oleksanf/vmshare/data/hrc_1.1/plink_maf0p001/chr2.bim', delim_whitespace=True, header=None, names='CHR SNP GP BP A1 A2'.split())

lib=libbgmg.LibBgmg('../src/build/lib/libbgmg.so', init_log='/home/oleksanf/vmshare/data/hrc_1.1/plink_maf0p001/test.log', context_id=0, dispose=True)
lib.defvec = bim['CHR'].notnull()
lib.chrnumvec = bim['CHR'].astype(int).values

r2min = 0.0499 # 0.05
ldscore_r2min = 0.01 # 0.001

lib.set_ld_r2_coo_from_file(2, '/home/oleksanf/vmshare/data/hrc_1.1/plink_maf0p001/chr2.run3.ld')
lib.set_ld_r2_csr()
lib_hetvec = 2*np.multiply(lib.mafvec, 1-lib.mafvec)

[snp, tag, lib_r] = lib.get_ld_r2_chr(2)
mafvec=lib.mafvec

#
#G=pandas_plink.read_plink1_bin('chr2.bed')
#v=G.sel(variant="2_2:10649:T:G").values
#w=G.sel(variant="2_2:29992862:C:T").values

In [ ]:
(array([ 1031, 11052, 11107, 11222, 19520, 19550, 19590, 19708, 19785, 19860, 19877, 19881, 19887]),)

In [4]:
idx1 = ((snp<146342) & (snp>103729) & (tag==1))  # 20 to 30 MB distance
idx3 = ((snp<146342) & (snp>103729) & (tag==3))  # 20 to 30 MB distance
set(snp[idx1]).difference(set(snp[idx3]))


{109558, 109729}

In [ ]:
import pandas_plink
pandas_plink.read_plink1_bin('', bim=None, fam=None, verbose=True)

In [10]:
lib_r[(snp==1)&(tag==3)]

array([0.96847486], dtype=float32)

In [15]:
offset=200000
for i in range(offset, offset+10):
    print(i, np.sum((snp<i+146342) & (snp>i+103729) & (tag==i)), 1-mafvec[i], bim.SNP[i], bim.SNP[np.max(snp[tag==i])])

200000 0 0.4328802824020386 2:39884087:A:T 2:40080343:G:A
200001 0 0.007232546806335449 2:39884670:C:G 2:40315460:C:T
200002 0 0.42493242025375366 2:39884712:G:A 2:40080343:G:A
200003 0 0.4240184426307678 2:39884791:A:G 2:40080343:G:A
200004 0 0.006914615631103516 2:39884935:G:C 2:45996694:A:G
200005 0 0.0031393766403198242 2:39884991:T:A 2:40341300:C:T
200006 0 0.4246940016746521 2:39885994:A:G 2:40080343:G:A
200007 0 0.001251816749572754 2:39886191:C:G 2:40244163:C:T
200008 423 0.001013338565826416 2:39886515:A:T 2:69841721:T:C
200009 0 0.001271665096282959 2:39887115:C:T 2:40191236:C:T


In [14]:
bim.SNP[np.max(snp[tag==200008])]

'2:69841721:T:C'

In [ ]:
423 0.001013338565826416 2:39886515:A:T
210 0.0013511180877685547 2:27332:C:T